In [6]:
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import os

from NeuralNetworkProj.Layers.ActivationFunctionLayers import ReLu,Sigmoid,Softmax
from NeuralNetworkProj.Layers.LinearLayer import LinearLayer
from NeuralNetworkProj.Layers.LossFuntionLayers import CrossEntropy
from NeuralNetworkProj.Utilities.DataTools import get_mnist_data,convert_dataset_numpy_array
from NeuralNetworkProj.Utilities.InitalizationFunctions import normal_initializer
from NeuralNetworkProj.Optim.LearningRate import LearningRate
from NeuralNetworkProj.Optim.Optimizer import NaiveUpdator

In [19]:
class Model():
    def __init__(self,model,updator) -> None:
        self.model=model
        self.updator=updator
    def __call__(self, x):
        input_=x
        for layer in self.model:
            input_=layer(input_)
        return input_

    def forward(self,x): # No recording method
        input_=x
        for layer in self.model:
            input_=layer.forward(input_)
        return input_

    def backward(self,grad):
        for layer in self.model[::-1]:
            grad=layer.backward(grad)
        return grad
    def Update(self):
        for layer in self.model:
            if not layer.require_update:
                continue
            seters=layer.parameter_seters()
            for name in layer.parameter_names():
                var_before=getattr(layer,name)
                grad_before=getattr(layer,name+"_grad")
                seters[name](self.updator(getattr(layer,name),getattr(layer,name+"_grad")))
                assert(np.allclose(getattr(layer,name),self.updator(var_before,grad_before)))
    
    def predict_prob(self,x):
        input_=x
        for layer in self.model:
            input_=layer.forward(input_) # avoid record data
        return input_
    
    def predict_label(self,x):
        prob=self.predict_prob(x)
        return np.argsort(prob,axis=1)[:,-1]

    def save_model(self,path):
        step=0
        folder=Path(path)
        if not folder.exists():
            folder.mkdir(parents=True)
        for layer in self.model:
            if layer.require_update:
                layer.save_model(str(folder/("layer_"+str(step))))
            step+=1
    def load_model(self,path):
        step=0
        for layer in self.model:
            if not layer.require_update:
                step+=1
                continue
            seters=layer.parameter_seters()
            for name in layer.parameter_names():
                temp=np.load(os.path.join(path,"layer_{}_{}.npy".format(step,name)))
                seters[name](temp)
            step+=1

In [20]:
## Define network structure
in_feature=784
hidden_feature_1=40
hidden_feature_2=20
out_feature=10

bias_option=True

netowrk=[
    LinearLayer(input_size=in_feature,output_size=hidden_feature_1,initialization=normal_initializer,bias=bias_option),
    ReLu(),
    LinearLayer(input_size=hidden_feature_1,output_size=hidden_feature_2,initialization=normal_initializer,bias=bias_option),
    Sigmoid(),
    LinearLayer(input_size=hidden_feature_2,output_size=out_feature,initialization=normal_initializer,bias=bias_option),
    Softmax()
]

loss_criteria=CrossEntropy()




In [21]:
model=Model(netowrk,NaiveUpdator(LearningRate(0.01)))

In [22]:
model.load_model("result/model_epoch200")

In [23]:
train_data,test_data=get_mnist_data()
trn_data_label=convert_dataset_numpy_array(train_data)
# trn_data_label[0]=trn_data_label[0].reshape(-1,784)

tst_data_label=convert_dataset_numpy_array(test_data)
# tst_data_label[0]=tst_data_label[0].reshape(-1,784)

In [27]:
tst_data_label[1].shape

(10000,)

In [28]:
prob=model.predict_prob(trn_data_label[0].reshape(-1,784))

In [39]:
np.sum(np.argsort(prob,axis=1)[:,-1]==trn_data_label[1].ravel())/len(trn_data_label[0])

0.7578

In [40]:
prob2=model.predict_prob(tst_data_label[0].reshape(-1,784))

In [42]:
np.sum(np.argsort(prob2,axis=1)[:,-1]==tst_data_label[1].ravel())

7622